# Minimum Wage Simulation -- Estimator Comparisons



In [ ]:
import os
# set wd to Users/boyuchen/Documents/UBC/RA/minimum_wage
os.chdir('/Users/boyuchen/Documents/UBC/RA/minimum_wage')

from codes.simulation_functions import *
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF



**Purpose**: demonstrate whether the three methods worked as expected.

**TL;DR:** 

1. Scenarios of choice: two truncated scenarios with no bunching and spillovers; then one with bunching and one with spillover.
2. Cengiz et al(2019) did a good job in not producing spurious spillover effect.
3. Percentile method gives the spurious "spillover effect" in S1 to S3.
4. My hazard estimates results have significant effect on bins above the minimum wage bin, which is not what I expected. 

In [ ]:
log_wages = generate_log_wages(1_000_000)  
# run df_grouped for four proability scenarios:
# real_m_pre, real_m_post, P_o, P_b, P_s
scenarios = {
    'S1': (12, 14, 0.5, 0, 0),
    'S2': (12, 14, 0.5, 0.1, 0),
    'S3': (12, 14, 0.5, 0, 0.3),
    'S4': (14, 16, 0.5, 0, 0),
    'S5': (14, 16, 0.5, 0.1, 0),
    'S6': (14, 16, 0.5, 0, 0.3)
}

# scenarios = {
#     'S1': (10.05, 12.05, 0.3, 0.5, 0),
#     'S2': (10.05, 12.05, 0.3, 0, 0.5),
#     'S3': (13.05, 15.05, 0.3, 0.5, 0),
#     'S4': (13.05, 15.05, 0.3, 0, 0.5),
# }

## Naive percentile OLS


In [ ]:
def calculate_percentiles(log_wages, scenario):
    real_m_pre, real_m_post, P_o, P_b, P_s = scenario
    df_pre = impose_minimum_wage(log_wages, real_m_pre, P_o, P_b, P_s)
    df_post = impose_minimum_wage(log_wages, real_m_post, P_o, P_b, P_s)


    def to_real_scale(log_wages):
        return np.exp(log_wages)

    df_pre['real_wages'] = to_real_scale(df_pre['adjusted_log_wages'])
    df_post['real_wages'] = to_real_scale(df_post['adjusted_log_wages'])

    percentiles = np.arange(0, 85, 5)

    pre_percentiles = np.percentile(df_pre['real_wages'].dropna(), percentiles)
    post_percentiles = np.percentile(df_post['real_wages'].dropna(), percentiles)
    
    # Calculate 95th percentile
    p95_pre = np.percentile(df_pre['real_wages'].dropna(), 95)
    p95_post = np.percentile(df_post['real_wages'].dropna(), 95)
    x_limit = max(p95_pre, p95_post)

    return pre_percentiles, post_percentiles, df_pre['real_wages'].dropna(), df_post['real_wages'].dropna(), x_limit


The figure shows the relative changes in precentiles as the minimum wage changes. As expected, all percentiles "inflated" as some more people got unemployed. We got bad results as expected.

In [ ]:
# Define the figure and axes for subplots
fig, axes = plt.subplots(2, 3, figsize=(16, 8), sharey=True)

# Flatten axes for easy iteration
axes = axes.flatten()

# Generate and plot results for each scenario
for i, (scenario_name, params) in enumerate(scenarios.items()):
    pre_percentiles, post_percentiles, _, _, x_limit = calculate_percentiles(log_wages, params)
    relative_changes = (post_percentiles - pre_percentiles) / pre_percentiles * 100

    ax = axes[i]
    percentiles = np.arange(5, 85, 5)


    ax.bar(percentiles, relative_changes[1:], width=4.5, alpha=0.7, color='green')
    ax.set_xlabel('Percentiles')
    # ax.set_ylabel('Relative Change (%)')
    ax.set_title(f'{scenario_name}: min wage {params[0]} to {params[1]}, $P_o$={params[2]}, $P_b$={params[3]}, $P_s$={params[4]}')
    ax.set_xticks(percentiles)
    ax.set_xlim([-5, 85])
    ax.set_ylim([min(relative_changes[1:]) - 5, max(relative_changes[1:]) + 5])

# Set the main title
plt.suptitle('Relative Changes in Wage Percentiles Across Scenarios', fontsize=16)
fig.text(0.04, 0.5, 'Relative Change (%)', va='center', rotation='vertical')

# Adjust layout
plt.tight_layout(rect=[0.04, 0.04, 1, 0.96])
plt.show()


## Cengiz et al.(2019) Method

I replicated and simplified Cengiz method. In S1 an S2, there are strong employment impact on bins right below the minimum wage bin. In S3, the method captures only bunching. In S4, the model captures the spillover effect. Overall the method produces good results that reflect the underlying model.  Bin size is 0.25.

*Notes: Since we use a two-period model, the old minimum wage  yields a visible trend near the level, which is not visible in their results since it has many period and states and the effect of "old" minimum wage is washed out.*

In [ ]:
# Function to apply Cengiz method for employment changes and wage effects
def cengiz_method(log_wages, scenario, no_minimum_wage=False):
    real_m_pre, real_m_post, P_o, P_b, P_s = scenario
    m_pre = np.log(real_m_pre)
    m_post = np.log(real_m_post)

    if no_minimum_wage:
        real_m_pre = 0
        
    # pass no_minimum_wage argument to impose_minimum_wage function
    df_pre = impose_minimum_wage(log_wages, real_m_pre, P_o, P_b, P_s)
    df_post = impose_minimum_wage(log_wages, real_m_post, P_o, P_b, P_s)

    df_pre['real_wages'] = np.exp(df_pre['adjusted_log_wages'])
    df_post['real_wages'] = np.exp(df_post['adjusted_log_wages'])
    
    # Define the 75 percentile of the pre-wage distribution (Wbar)
    Wbar = df_pre['real_wages'].quantile(0.75)

    bin_width = 0.1

    # Assign bins for pre and post data, excluding unemployed
    bins_pre = np.arange(0, Wbar + bin_width, bin_width)
    bins_post = np.arange(0, Wbar + bin_width, bin_width)

    df_pre['wage_bin'] = pd.cut(df_pre['real_wages'], bins=bins_pre, right=False)
    df_post['wage_bin'] = pd.cut(df_post['real_wages'], bins=bins_post, right=False)

    # Calculate the pre-period average wage of workers below the initial minimum wage
    affected_pre = df_pre[df_pre['original_log_wages'] < m_pre]
    # affected_post = df_post[df_post['original_log_wages'] < m_pre]

    average_affected_wage_pre = affected_pre['real_wages'].mean()
    affected_wage_bill_pre = affected_pre['real_wages'].sum()
    total_wage_bill_pre = df_pre['real_wages'].sum()
    total_wage_bill_post = df_post['real_wages'].sum()
    Δtotal_wage_bill = total_wage_bill_post - total_wage_bill_pre
    
    # Count the number of workers in each wage bin, excluding unemployed
    pre_counts = df_pre['wage_bin'].value_counts().sort_index()
    post_counts = df_post['wage_bin'].value_counts().sort_index()

    if no_minimum_wage:
        return {
            'pre_counts': pre_counts,
            'post_counts': post_counts,
            'bins_pre': pre_counts.index.tolist()
        }
    else:

        # count the total number of affected worker in the pre-period using affected_pre
        total_affected_pre = affected_pre.shape[0]

        pre_counts = pre_counts.reindex_like(post_counts).fillna(0)
        post_counts = post_counts.reindex_like(pre_counts).fillna(0)

        # Create a boolean mask for intervals where the left edge is greater than or equal to m_post
        mask_post = np.array([interval.left >= np.exp(m_post).round(3) for interval in post_counts.index])
        mask_pre = np.array([interval.left >= np.exp(m_post).round(3) for interval in pre_counts.index])

        # Calculate excess and missing jobs, only considering bins below Wbar
        Δa = post_counts[mask_post].sum() - pre_counts[mask_pre].sum()
        Δb = post_counts[~mask_post].sum() - pre_counts[~mask_pre].sum()
        Δemp = Δa + Δb

        total_employed_pre = pre_counts.sum()

        Δm = real_m_post - real_m_pre
        employment_elasticity = (Δemp / total_employed_pre) / (Δm / real_m_pre)

        # change in total employment over the "affected workers" in the pre-period
        affected_employment_change = Δemp / total_affected_pre

        # change in wage bills of affected workers is the pre-period affected wage bill + the total change in wage bill of all workers
        affected_new_wage_bill = affected_wage_bill_pre + Δtotal_wage_bill
        affected_employment_post = total_affected_pre + Δemp

        # calculate percentage change in the average wage of affected workers
        new_average_wage_affected = affected_new_wage_bill / affected_employment_post

        # calculate percentage change in the average wage of affected workers
        percentage_change_average_wage_affected = (new_average_wage_affected - average_affected_wage_pre) / average_affected_wage_pre

        # calculate own_elasticity 
        own_wage_elasticity = affected_employment_change / percentage_change_average_wage_affected

        return {
            'employment_elasticity': employment_elasticity,
            'own_wage_elasticity': own_wage_elasticity,
            'percentage_change_affected_employment': affected_employment_change,
            'percentage_change_average_wage_affected': percentage_change_average_wage_affected,
            'Δa': Δa,
            'Δb': Δb,
            'bins_pre': pre_counts.index.tolist(),
            'pre_counts': pre_counts,
            'post_counts': post_counts
        }



In [ ]:
def plot_changes_in_employment(result, scenario_name, scenario, no_minimum_wage=False):
    real_m_pre, real_m_post, P_o, P_b, P_s = scenario
    pre_counts = result['pre_counts']
    post_counts = result['post_counts']
    bins_pre = result['bins_pre']

    if no_minimum_wage==False:

        Δa = result['Δa']
        Δb = result['Δb']
        percentage_change_affected_employment = result['percentage_change_affected_employment']
        percentage_change_average_wage_affected = result['percentage_change_average_wage_affected']

    # Calculate the changes in employment
    employment_changes = post_counts - pre_counts

    # Calculate the total pre-employment
    total_pre_employment = pre_counts.sum()

    # Calculate relative changes to pre-total employment
    relative_employment_changes = employment_changes / total_pre_employment * 100

    # Extract intervals for display
    intervals = [interval.left for interval in bins_pre]

    # Calculate bins relative to real_m_post
    bins_relative_to_post = [round((interval - real_m_post) / (bins_pre[1].left - bins_pre[0].left)) for interval in intervals]

    # Find the positions for m_pre and m_post
    m_pre_pos = next(i for i, interval in enumerate(intervals) if interval >= real_m_pre)
    m_post_pos = next(i for i, interval in enumerate(intervals) if interval >= real_m_post)

    # Select 10 bins below real_m_pre and 30 bins above real_m_post
    start_idx = max(0, m_pre_pos - 4)
    end_idx = min(len(intervals), m_post_pos + 20)

    bins_relative_to_post = bins_relative_to_post[start_idx:end_idx]
    relative_employment_changes = relative_employment_changes[start_idx:end_idx]

    bar_width = 0.5
    index = np.arange(len(bins_relative_to_post))

    ax.bar(index, relative_employment_changes, bar_width, alpha=0.7, color='green', label='Change in Employment')

    ax.set_xlabel('Bins relative to new MW')
    # ax.set_ylabel('% Change in Employment Relative to Pre-Total Employment')
    ax.set_title(f'{scenario_name}: minimum wage from {real_m_pre} to {real_m_post}, $P_o$={P_o}, $P_b$={P_b}, $P_s$={P_s}')
    ax.set_xticks(index[::2])
    ax.set_xticklabels(bins_relative_to_post[::2], rotation=90)

    # Add grey line at y=0
    ax.axhline(y=0, color='grey', linestyle='-', linewidth=1)

    # Adjust positions for m_pre and m_post lines to match the selected bins
    m_pre_pos_adj = m_pre_pos - start_idx
    m_post_pos_adj = m_post_pos - start_idx

    ax.set_ylim(-5, 5)

    if no_minimum_wage==False:
        # Add m_pre and m_post lines
        ax.axvline(m_pre_pos_adj - 0.5, color='blue', linestyle='--', linewidth=2, label='original minimum wage')
        ax.axvline(m_post_pos_adj - 0.5, color='red', linestyle='--', linewidth=2, label='new minimum wage')

        # Add parameters to the lower left
        ax.text(0.62, 0.1,
                f'Δa = {Δa:.3f}\nΔb = {Δb:.3f}\n%Δ affected emp. = {percentage_change_affected_employment:.3f}\n%Δ affected wage = {percentage_change_average_wage_affected:.3f}',
                bbox=dict(facecolor='white', alpha=0.5), transform=ax.transAxes)

    # ax.legend()
    plt.tight_layout()

# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(16, 8))

# Generate and plot results for each scenario
for ax, (scenario_name, scenario) in zip(axs.flatten(), scenarios.items()):
    result = cengiz_method(log_wages, scenario, no_minimum_wage=True)
    plot_changes_in_employment(result, scenario_name, scenario, no_minimum_wage=True)

# Set common ylabel
fig.text(0.04, 0.5, 'Percentage Change in Employment Relative to Pre-Total Employment (%)', va='center', rotation='vertical')
plt.suptitle('Effect of the Minimum Wage on the Wage Distribution', fontsize=16)
# only show legend for panel 3
axs[0, 1].legend()

# Adjust layout
plt.tight_layout(rect=[0.04, 0.04, 1, 0.96])
plt.show()


In [ ]:
def prepare_cengiz_data(log_wages, scenario):
    real_m_pre, real_m_post, P_o, P_b, P_s = scenario

    total_population = log_wages.shape[0]

    # Assume impose_minimum_wage is defined and applies the minimum wage policy to the data
    df_pre = impose_minimum_wage(log_wages, real_m_pre, P_o, P_b, P_s)
    df_post = impose_minimum_wage(log_wages, real_m_post, P_o, P_b, P_s)

    df_pre['real_wages'] = np.exp(df_pre['adjusted_log_wages'])
    df_post['real_wages'] = np.exp(df_post['adjusted_log_wages'])

    # Simplify bins
    bins = [0] + list(np.arange(3, 4, 0.5)) + list(np.arange(4, 20, 0.1)) + [20, np.inf]

    def create_min_dummies(df, real_m):
        # Create bins and labels
        df['wagcat'] = pd.cut(df['real_wages'], bins=bins, labels=False, duplicates='drop')
        # Change wagcat to be integer or NaN
        df['wagcat'] = df['wagcat'].astype('Int64')

        # Find the bin number containing the minimum wage
        m_bin = pd.cut([real_m], bins=bins, labels=False, duplicates='drop')[0]

        # Aggregate data by bin
        df_grouped = df.groupby('wagcat').size().reset_index(name='fweight')
        df_grouped['normalized_emp'] = df_grouped['fweight'] / total_population
        max_bin = df_grouped['wagcat'].max()
        all_bins = pd.DataFrame({'wagcat': range(max_bin + 1)})
        df_grouped = pd.merge(all_bins, df_grouped, on='wagcat', how='left').fillna(0)

        # Create min and variables that the relative position to the minimum wage
        df_grouped['rel_pos'] = df_grouped['wagcat'] - m_bin

        # Create bin* dummies
        bin_dummies = pd.get_dummies(df_grouped['wagcat'], prefix='bin')

        for i in range(20):
            # create min* dummies
            df_grouped[f'min{i+1}b'] = (df_grouped['rel_pos'] == -(i+1)).astype(int)

        for i in range(20):
            # create min* dummies
            df_grouped[f'min{i+1}a'] = (df_grouped['rel_pos'] == i+1).astype(int)

        df_grouped['min0'] = df_grouped['rel_pos'] == 0

        df_grouped = pd.concat([df_grouped, bin_dummies], axis=1)
        
        return df_grouped
    
    # Create min dummies and group by bin-period for both pre and post data
    df_grouped_pre = create_min_dummies(df_pre, real_m_pre)
    df_grouped_pre['period'] = 'pre'

    df_grouped_post = create_min_dummies(df_post, real_m_post)
    df_grouped_post['period'] = 'post'

    # Combine pre and post data
    df_grouped = pd.concat([df_grouped_pre, df_grouped_post])
    df_grouped = df_grouped.astype({col: 'int64' if df_grouped[col].dtype == 'bool' else df_grouped[col].dtype for col in df_grouped.columns})
    df_grouped = sm.add_constant(df_grouped)

    # drop lowest 5 and top 5 bins
    df_grouped = df_grouped[(df_grouped['wagcat'] >= 5) & (df_grouped['wagcat'] <= 160)]

    # Sanity check 1: sum of min* variables must be 0 or 1
    min_vars = [col for col in df_grouped.columns if col.startswith('min')]
    df_grouped['min_sum'] = df_grouped[min_vars].sum(axis=1)
    assert df_grouped['min_sum'].isin([0, 1]).all(), "Sanity check failed: sum of min* variables is not 0 or 1 for some rows."

    # Sanity check 2: sum of bin* variables must be 1
    bin_vars = [col for col in df_grouped.columns if col.startswith('bin')]
    df_grouped['bin_sum'] = df_grouped[bin_vars].sum(axis=1)
    assert (df_grouped['bin_sum'] == 1).all(), "Sanity check failed: sum of bin* variables is not 1 for some rows."

    # drop min_sum and bin_sum
    df_grouped = df_grouped.drop(columns=['min_sum', 'bin_sum'])


    return df_grouped

In [ ]:
def plot_cengiz_ols(log_wages, scenario):
    real_m_pre, real_m_post, P_o, P_b, P_s = scenario
    data = prepare_cengiz_data(log_wages, scenario)
    
    real_m_pre, real_m_post, P_o, P_b, P_s = scenario
    data = prepare_cengiz_data(log_wages, scenario)
    # Prepare the independent variables (min* and bin* dummies)
    X = data.filter(regex='^(min\\d{1,2}|bin)').copy()
    y = data['normalized_emp']

    # Fit the OLS model
    model = sm.OLS(y, X).fit()
    # Generate and plot results for each scenario
    model.params.filter(regex='^(min\\d{1,2}|min0)')

    paras = model.params.filter(regex='^(min\\d{1,2}|min0)')
    coef_df = pd.DataFrame(paras).reset_index()
    # rename column "0" as "coefficient"
    coef_df = coef_df.rename(columns={0: 'coefficient'})

    # for min[n]b, assign rel_pos = -n; for min0, assign rel_pos = 0; for min[n]a, assign rel_pos = n
    coef_df['rel_pos'] = coef_df['index'].str.extract('(\d{1,2})').astype(int)
    coef_df['rel_pos'] = np.where(coef_df['index'].str.contains('a'), coef_df['rel_pos'], -coef_df['rel_pos'])
    # sort coef_df by rel_pos
    coef_df = coef_df.sort_values('rel_pos')

    # plot bar plot
    ax.figsize=(12, 6)
    ax.bar(coef_df['rel_pos'], coef_df['coefficient'], color='green', alpha=0.7)
    ax.set_xlabel('Bins relative to new MW')
    ax.set_ylabel('Coefficient')
    ax.set_ylim(-0.01, 0.03)
    ax.set_title(f'{scenario_name}: minimum wage from {real_m_pre} to {real_m_post}, $P_o$={P_o}, $P_b$={P_b}, $P_s$={P_s}')


In [ ]:
# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(16, 8))

# Generate and plot results for each scenario
for ax, (scenario_name, scenario) in zip(axs.flatten(), scenarios.items()):
    plot_cengiz_ols(log_wages, scenario)
    
# Set common ylabel
fig.text(0.04, 0.5, 'Percentage Change in Employment Relative to Pre-Total Employment (%)', va='center', rotation='vertical')
plt.suptitle('Effect of the Minimum Wage on the Wage Distribution', fontsize=16)
# only show legend for panel 3
axs[0, 1].legend()

# Adjust layout
plt.tight_layout(rect=[0.04, 0.04, 1, 0.96])
plt.show()


In [ ]:
def plot_cengiz_counterfactual(log_wages, scenario):
    real_m_pre, real_m_post, P_o, P_b, P_s = scenario
    data = prepare_cengiz_data(log_wages, scenario)
    
    # Prepare the independent variables (min* and bin* dummies)
    X = data.filter(regex='^(min\\d{1,2}|bin)').copy()
    y = data['normalized_emp']

    # Fit the OLS model
    model = sm.OLS(y, X).fit()
    # Generate and plot results for each scenario
    model.params.filter(regex='^(min\\d{1,2}|min0)')

    # Create a copy of the original data
    df_counterfactual = data.copy()
    # Identify columns starting with 'min'
    min_columns = [col for col in df_counterfactual.columns if col.startswith('min')]
    # Set these columns to 0
    df_counterfactual[min_columns] = 0 
    # predict counterfactual employment
    df_counterfactual['pred_emp'] = model.predict(df_counterfactual[X.columns])
    # calculate the difference between actual and predicted employment using post period data
    # tot_emp_pre = df_counterfactual[df_counterfactual['period'] == 'pre']['normalized_emp'].sum()
    df_counterfactual_post = df_counterfactual[df_counterfactual['period'] == 'post']
    df_counterfactual_post['emp_diff'] = df_counterfactual_post['normalized_emp'] - df_counterfactual_post['pred_emp']

    # sort from lowest bin
    df_counterfactual_post = df_counterfactual_post.sort_values('rel_pos')

    # running sum of emp change 
    df_counterfactual_post['emp_diff_sum'] = df_counterfactual_post['emp_diff'].cumsum()

    # find rows with largest emp_diff
    df_counterfactual_post.nlargest(10, 'emp_diff')

    ax.bar(df_counterfactual_post['rel_pos'], df_counterfactual_post['emp_diff'], width=1, color='blue', alpha=0.7, label='Change in Employment')

    ax.set_xlabel('Bins relative to new MW')
    # ax.set_ylabel('% Change in Employment Relative to Pre-Total Employment')
    ax.set_title(f'{scenario_name}: minimum wage from {real_m_pre} to {real_m_post}, $P_o$={P_o}, $P_b$={P_b}, $P_s$={P_s}')
    ax.set_xlim(-20, 20)
    plt.tight_layout()


In [ ]:
# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(16, 8))

# Generate and plot results for each scenario
for ax, (scenario_name, scenario) in zip(axs.flatten(), scenarios.items()):
    plot_cengiz_counterfactual(log_wages, scenario)

# Set common ylabel
fig.text(0.04, 0.5, 'Percentage Change in Employment Relative to Pre-Total Employment (%)', va='center', rotation='vertical')
plt.suptitle('Effect of the Minimum Wage on the Wage Distribution', fontsize=16)
# only show legend for panel 3
axs[0, 1].legend()

# Adjust layout
plt.tight_layout(rect=[0.04, 0.04, 1, 0.96])
plt.show()


## Hazard Estimator


I first prepare the data and variables following the definition that Thomas gave me.

In [ ]:
def prepare_hazard_var(log_wages, real_m_pre, real_m_post, P_o=0.2, P_b=0, P_s=0.5, append_unaffected_wages = False):

    # Assuming impose_minimum_wage function is defined elsewhere
    df_pre = impose_minimum_wage(log_wages, real_m_pre, P_o, P_b, P_s)
    df_post = impose_minimum_wage(log_wages, real_m_post, P_o, P_b, P_s)
    
    df_pre['real_wages'] = np.exp(df_pre['adjusted_log_wages'])
    df_post['real_wages'] = np.exp(df_post['adjusted_log_wages'])

    # drop the unemployed
    df_pre = df_pre.dropna(subset=['real_wages'])
    df_post = df_post.dropna(subset=['real_wages'])

    # Simplify bins
    bins = [0] + list(np.arange(3, 4, 0.5)) + list(np.arange(4, 20, 0.1)) + [20, np.inf]
    min_vars = ['min', 'min21b', 'min1620b', 'min1115b','min610b', 'min35b', 'min12b', 'min12a', 'min35a', 'min610a', 'min1115a', 'min1620a']

    def create_min_dummies(df, real_m):
        # Create bins and labels
        df['wagcat'] = pd.cut(df['real_wages'], bins=bins, labels=False, duplicates='drop')
        # Change wagcat to be integer or NaN
        df['wagcat'] = df['wagcat'].astype('Int64')

        # Find the bin number containing the minimum wage
        m_bin = pd.cut([real_m], bins=bins, labels=False, duplicates='drop')[0]

        # Aggregate data by bin
        df_grouped = df.groupby('wagcat').size().reset_index(name='fweight')
        max_bin = df_grouped['wagcat'].max()
        all_bins = pd.DataFrame({'wagcat': range(max_bin + 1)})
        df_grouped = pd.merge(all_bins, df_grouped, on='wagcat', how='left').fillna(0)

        # Create min and min* variables
        df_grouped['min'] = np.where(df_grouped['wagcat'] == m_bin, 1, 0)
        df_grouped['min21b'] = np.where((df_grouped['wagcat'] <= m_bin - 21), 1, 0)
        df_grouped['min1620b'] = np.where((df_grouped['wagcat'] >= m_bin - 20) & (df_grouped['wagcat'] <= m_bin - 16), 1, 0)
        df_grouped['min1115b'] = np.where((df_grouped['wagcat'] >= m_bin - 15) & (df_grouped['wagcat'] <= m_bin - 11), 1, 0)
        df_grouped['min610b'] = np.where((df_grouped['wagcat'] >= m_bin - 10) & (df_grouped['wagcat'] <= m_bin - 6), 1, 0)
        df_grouped['min35b'] = np.where((df_grouped['wagcat'] >= m_bin - 5) & (df_grouped['wagcat'] <= m_bin - 3), 1, 0)
        df_grouped['min12b'] = np.where((df_grouped['wagcat'] >= m_bin - 2) & (df_grouped['wagcat'] <= m_bin - 1), 1, 0)
        df_grouped['min12a'] = np.where((df_grouped['wagcat'] >= m_bin + 1) & (df_grouped['wagcat'] <= m_bin + 2), 1, 0)
        df_grouped['min35a'] = np.where((df_grouped['wagcat'] >= m_bin + 3) & (df_grouped['wagcat'] <= m_bin + 5), 1, 0)
        df_grouped['min610a'] = np.where((df_grouped['wagcat'] >= m_bin + 6) & (df_grouped['wagcat'] <= m_bin + 10), 1, 0)
        df_grouped['min1115a'] = np.where((df_grouped['wagcat'] >= m_bin + 11) & (df_grouped['wagcat'] <= m_bin + 15), 1, 0)
        df_grouped['min1620a'] = np.where((df_grouped['wagcat'] >= m_bin + 16) & (df_grouped['wagcat'] <= m_bin + 20), 1, 0)
        df_grouped['min_bin'] = m_bin
        df_grouped['rel_min_bin'] = df_grouped['wagcat'] - m_bin

        # Create bin* dummies
        bin_dummies = pd.get_dummies(df_grouped['wagcat'], prefix='bin')
        df_grouped = pd.concat([df_grouped, bin_dummies], axis=1)
        
        return df_grouped

    # Create min dummies and group by bin-period for both pre and post data
    df_grouped_pre = create_min_dummies(df_pre, real_m_pre)
    df_grouped_pre['period'] = 'pre'

    df_grouped_post = create_min_dummies(df_post, real_m_post)
    df_grouped_post['period'] = 'post'

    # Combine pre and post data
    df_grouped = pd.concat([df_grouped_pre, df_grouped_post])

    if append_unaffected_wages:
        df_raw = pd.DataFrame()
        df_raw['real_wages'] = np.exp(log_wages)
        
        # Create bins and labels
        df_raw['wagcat'] = pd.cut(df_raw['real_wages'], bins=bins, labels=False, duplicates='drop')
        # Change wagcat to be integer or NaN
        df_raw['wagcat'] = df_raw['wagcat'].astype('Int64')
        
        # Aggregate data by bin
        df_grouped_raw = df_raw.groupby('wagcat').size().reset_index(name='fweight')
        # define all var in min_vars to be 0
        df_grouped_raw[min_vars] = 0
        # Create bin* dummies
        bin_dummies = pd.get_dummies(df_grouped_raw['wagcat'], prefix='bin')
        df_grouped_raw = pd.concat([df_grouped_raw, bin_dummies], axis=1)
        df_grouped_raw['period'] = 'counterfactual'
        df_grouped = pd.concat([df_grouped, df_grouped_raw])

    max_bin = df_grouped['wagcat'].max()


    # Calculate remain: cumulative sum of fweight in descending order of bins
    df_grouped = df_grouped.sort_values(by=['period', 'wagcat'], ascending=[True, False])
    df_grouped['remain'] = df_grouped.groupby('period')['fweight'].cumsum()
    df_grouped['hazard'] = df_grouped['fweight'] / df_grouped['remain']

    # assert that by group, remain - remain.shift(1) == fweight
    df_grouped = df_grouped.sort_values(by=['period', 'wagcat'])

    # # group by period: df_grouped['remain_diff'] = df_grouped['remain'].diff()
    df_grouped['remain_diff'] = df_grouped.groupby('period')['remain'].diff()
    df_grouped['lag_fweight'] = df_grouped.groupby('period')['fweight'].shift(1)

    # check if df_grouped[(df_grouped['lag_fweight'] + df_grouped['remain_diff'] == 0) | df_grouped['remain_diff'].isna()].shape[0] == df_grouped.shape[0]
    assert (df_grouped[(df_grouped['lag_fweight'] + df_grouped['remain_diff'] == 0) | df_grouped['remain_diff'].isna()].shape[0] == df_grouped.shape[0]), "Sanity check failed: remain - remain.shift(1) != fweight for some rows"

    # check fweight > 0 for all rows
    assert (df_grouped['fweight'] > 0).all(), "Sanity check failed: fweight <= 0 for some rows"

    # generate ln(-ln(1 - h(y))) as cloglog_hazard
    df_grouped['cloglog_hazard'] = np.log(-np.log(1 - df_grouped['fweight'] / df_grouped['remain']))

    # check cloglog_hazard is not NaN if wagcat is not 161
    assert df_grouped[df_grouped['wagcat'] != max_bin]['cloglog_hazard'].notna().all(), "Sanity check failed: cloglog_hazard is NaN for some rows where wagcat != 161"
    #drop wagcat == 161
    df_grouped = df_grouped[df_grouped['wagcat'] != max_bin]

    # Sanity check 1: sum of min* variables must be 0 or 1
    df_grouped['min_sum'] = df_grouped[min_vars].sum(axis=1)
    assert df_grouped['min_sum'].isin([0, 1]).all(), "Sanity check failed: sum of min* variables is not 0 or 1 for some rows."

    # Sanity check 2: sum of bin* variables must be 1
    bin_vars = [col for col in df_grouped.columns if col.startswith('bin')]
    df_grouped['bin_sum'] = df_grouped[bin_vars].sum(axis=1)
    assert (df_grouped['bin_sum'] == 1).all(), "Sanity check failed: sum of bin* variables is not 1 for some rows."

    # drop bin_sum and min_sum columns
    df_grouped = df_grouped.drop(columns=['bin_sum', 'lag_fweight', 'remain_diff'])
    
    return df_grouped

Create data for four scenarios.

In [ ]:
df_appended = pd.DataFrame()

# loop through the scenarios
for scenario, params in scenarios.items():
    real_m_pre, real_m_post, P_o, P_b, P_s = params
    df_grouped = prepare_hazard_var(log_wages, real_m_pre, real_m_post, P_o, P_b, P_s)
    df_grouped['Scenario'] = scenario
    df_appended = pd.concat([df_appended, df_grouped])
    print(f"Scenario: {scenario}")
    # print #rows fweight ==0
    # print(df_grouped[df_grouped['fweight']==0].shape[0])
    df_grouped.to_stata(f'data/df_grouped_{scenario}.dta')

In [ ]:
#barplot y = fweight and remain ; x = wagcat for each scenario of df_appended. 
df_appended['remain'] = df_appended['remain'].astype(float)
df_appended['fweight'] = df_appended['fweight'].astype(float)

# Create a figure and axes for subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharey=True)

# Flatten axes for easy iteration
axes = axes.flatten()

# Plot each scenario in a separate subplot
for i, scenario in enumerate(scenarios):
    real_m_pre, real_m_post, P_o, P_b, P_s = scenarios[scenario]
    df = df_appended[df_appended['Scenario']==scenario]
    pre_minimum_wage_bin = df[(df['min'] == 1) & (df['period'] == "pre")]['wagcat'].values[0]
    post_minimum_wage_bin = df[(df['min'] == 1) & (df['period'] == "post")]['wagcat'].values[0]

    # Plot the data
    axes[i].bar(df[df['period']=='pre']['wagcat'], df[df['period']=='pre']['fweight'], capsize=5, label=scenario, color = 'green', alpha = 0.5)
    axes[i].bar(df[df['period']=='post']['wagcat'], df[df['period']=='post']['fweight'], capsize=5, label=scenario, color = 'blue', alpha = 0.5)
    # add vline for minimum wage to bar
    axes[i].axvline(x=pre_minimum_wage_bin, color='green', linestyle='--', linewidth=2)
    axes[i].axvline(x=post_minimum_wage_bin, color='blue', linestyle='--', linewidth=2)
    axes[i].legend(['pre', 'post'], loc='upper right')

    axes[i].set_xlabel('Wage bin')
    axes[i].set_title(f'{scenario}: minimum wage from {real_m_pre} to {real_m_post}, $P_o$={P_o}, $P_b$={P_b}, $P_s$={P_s}')
    # set ylim
    axes[i].set_ylim([0, 10_000])

# Set common ylabel
fig.text(0.04, 0.5, 'counts', va='center', rotation='vertical')
# Set the main title
plt.suptitle('Employment counts in each bin (fweight)', fontsize=16)

# Adjust layout
plt.tight_layout(rect=[0.04, 0.04, 1, 0.96])
plt.show()

Call STATA and run do.file.

In [ ]:
# %pip install stata_setup
import stata_setup
stata_setup.config("/Applications/Stata/", "se")

In [ ]:
%%stata
# execute the following do.file
do "codes/hazard simulation.do"

### Notes

- As shown in both the regression results above and figures below, we observed significant positive coefficient in the bins above the minimum wage, which is confusing since there shouldn't be any spillover effect in S1 to S3. 

- I'm thinking if it has anything to do with the "binomial" assumption. Does the distribution assumption impose `fweight` or `fweight`/`remain` to follow a bell shape, or it means something else? If my guess is correct, since the distribution of S1 and S2 are truncated bell shape, to match the bell shape of binomial we got negative coefficients in bins below minimum wage and positive coefficients for bins above, even for `min1620a`. 

- Does the issue only happen in my simulation context? How does the full model avoid this issue with real world data (many period and states.)


In [ ]:
# Define the x-axis ranges for the variables
# x_ranges = {
#     'min': (0, .1),
#     'min6b': (-2.0, -.5),
#     'min35b': (-.5, -.2),
#     'min12b': (-.2, .0),
#     'min12a': (.1, .3),
#     'min35a': (.3, .6),
#     'min610a': (.6, 1.1),
#     'min1115a': (1.1, 1.6),
#     'min1620a': (1.6, 2.0)
# }

x_ranges = {
    'min': (0, .1),
    'min610b': (-1.0, -.5),
    'min1115b': (-1.5, -1.0),
    'min1620b': (-2.0, -1.5),
    'min21b': (-2.5, -2.0),
    'min35b': (-.5, -.2),
    'min12b': (-.2, .0),
    'min12a': (.1, .3),
    'min35a': (.3, .6),
    'min610a': (.6, 1.1),
    'min1115a': (1.1, 1.6),
    'min1620a': (1.6, 2.0)
}

In [ ]:
# Load the Excel file
file_path = 'tables/glm_coefficients.xlsx'

# Function to format the data
def format_data(file_path, scenario):
    df = pd.read_excel(file_path, sheet_name=scenario, header=None)
    variables = df.iloc[1, 1:].values
    coefficients = df.iloc[2, 1:].values
    se = df.iloc[3, 1:].values
    data = pd.DataFrame({
        'Variable': variables,
        'Coefficient': coefficients,
        'SE': se,
        'Scenario': scenario
    })

    # only keep columns where variable start with "min"
    data = data[data['Variable'].str.startswith('min')==True]

    return data

glm_result = pd.concat([format_data(file_path, scenario) for scenario in scenarios.keys() ])

# Filter the data for the variables of interest
glm_result = glm_result[glm_result['Variable'].isin(x_ranges.keys())]


In [ ]:
# prepare df to overlay the glm coefficients
def prepare_fweight_df(df_appended, scenario):
    df_appended['rel_bin'] = df_appended['wagcat'] - df_appended['min_bin']
    df = df_appended[(df_appended['rel_bin'] >= -10) & (df_appended['rel_bin'] <= 20)]
    # rel_dollar is the relative bin * 10
    df['rel_dollar'] = df['rel_bin'] * 0.1
    df_pre = df[(df['period']=='pre') & (df['Scenario']==scenario)]
    df_post = df[(df['period']=='post') & (df['Scenario']==scenario)]
    df_pre = df_pre[['rel_dollar', 'fweight']]
    df_post = df_post[['rel_dollar', 'fweight']]
    df_pre.columns = ['rel_dollar', 'fweight_pre']
    df_post.columns = ['rel_dollar', 'fweight_post']
    df_fweight = pd.merge(df_pre, df_post, on='rel_dollar', how='outer')

    return df_fweight

# prepare_fweight_df(df_appended, 'S1')

In [ ]:
# Load the Excel file
file_path = 'tables/ols_coefficients.xlsx'

# Function to format the data
def format_data(file_path, scenario):
    df = pd.read_excel(file_path, sheet_name=scenario, header=None)
    variables = df.iloc[1, 1:].values
    coefficients = df.iloc[2, 1:].values
    se = df.iloc[3, 1:].values
    data = pd.DataFrame({
        'Variable': variables,
        'Coefficient': coefficients,
        'SE': se,
        'Scenario': scenario
    })

    # only keep columns where variable start with "min"
    data = data[data['Variable'].str.startswith('min')==True]

    return data

ols_result = pd.concat([format_data(file_path, scenario) for scenario in scenarios.keys() ])

# Filter the data for the variables of interest
ols_result = ols_result[ols_result['Variable'].isin(x_ranges.keys())]


Tried OLS and got the same coefficients. The SE from OLS are unadjusted.

In [ ]:
glm_result['Model'] = 'glm'
ols_result['Model'] = 'ols'
result_appended = pd.concat([glm_result, ols_result])

# Define the figure and axes for subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharey=True)

# Flatten axes for easy iteration
axes = axes.flatten()

# Plot each scenario in a separate subplot
for i, scenario in enumerate(scenarios):
    real_m_pre, real_m_post, P_o, P_b, P_s = scenarios[scenario]
    scenario_data_ols = result_appended[(result_appended['Scenario'] == scenario) & (result_appended['Model'] == 'ols')]
    scenario_data_glm = result_appended[(result_appended['Scenario'] == scenario) & (result_appended['Model'] == 'glm')]
    
    x_scenario_values_ols = [np.mean([x_ranges[var][0], x_ranges[var][1]]) for var in scenario_data_ols['Variable']]  # Convert to percentage
    y_values_ols = scenario_data_ols['Coefficient'] * 100  # Convert to percentage
    y_err_ols = scenario_data_ols['SE'] * 100  # Convert to percentage
    scenario_widths_ols = [(x_ranges[var][1] - x_ranges[var][0]) for var in scenario_data_ols['Variable']]  # Convert to percentage

    x_scenario_values_glm = [np.mean([x_ranges[var][0], x_ranges[var][1]]) for var in scenario_data_glm['Variable']]  # Convert to percentage
    y_values_glm = scenario_data_glm['Coefficient'] * 100  # Convert to percentage
    y_err_glm = scenario_data_glm['SE'] * 100  # Convert to percentage
    scenario_widths_glm = [(x_ranges[var][1] - x_ranges[var][0]) for var in scenario_data_glm['Variable']]  # Convert to percentage
    
    # Plot the OLS data
    axes[i].bar(x_scenario_values_ols, y_values_ols, width=scenario_widths_ols, yerr=y_err_ols, capsize=5, label='OLS', color='green', alpha=0.5)
    
    # Plot the GLM data with hollow bars 
    axes[i].bar(x_scenario_values_glm, y_values_glm, width=scenario_widths_glm, yerr=y_err_glm, capsize=5, label='GLM', color='white', alpha=0.5, edgecolor='black', linewidth=1.5, hatch='//')
    
    # add old and new minimum wage lines with legend
    # axes[i].axvline(x=real_m_pre - real_m_post, color='blue', linestyle='--', linewidth=2, label='Original minimum wage')
    # axes[i].axvline(x=0, color='red', linestyle='--', linewidth=2, label='New minimum wage')
         
    axes[i].set_xlabel('Dollars above the minimum wage')
    axes[i].set_title(f'{scenario}: minimum wage from {real_m_pre} to {real_m_post}, $P_o$={P_o}, $P_b$={P_b}, $P_s$={P_s}')
    
    # Set x-ticks with a break of 5 bins (0.5 dollars)
    axes[i].set_xticks(np.arange(-2.5, 2.1, .5))
    axes[i].set_xticklabels(np.arange(-2.5, 2.1, 0.5))

# Set common ylabel
fig.text(0.04, 0.5, 'Effect of the min. wage on wage bin probabilities (%)', va='center', rotation='vertical')

# Set the main title
plt.suptitle('Effect of the Minimum Wage on the Wage Distribution (OLS and GLM)', fontsize=16)

# Adjust layout
plt.tight_layout(rect=[0.04, 0.04, 1, 0.96])

# Add a legend
# handles, labels = axes[0].get_legend_handles_labels()
axes[1].legend()

# fig.legend(handles, labels, loc='upper right')

plt.show()

# Notes

- The hazard estimation and Cengiz et al.(2019) both shows the negative employment effect below the minimum wage, and null effect when the model has no bunching or spillover. However, the Cengiz et al.(2019) method largely depends on the numbers of employments in a given bin prior to the minimum wage change. In bins with more counts we will observe a larger negative impact (such as the original minimum wage bin). Such an effect was not observed in the hazard estimation.
- In the hazard estimation, using the GLM and OLS gives the exactly same coefficients. The hazard estimation captures the "drop" for all cells below new minimum wage.
- The naive approach that compares the wage percentile of employed workers captures fallacious spillover effects when there are not (See S1, S2, and S3)